<DIV ALIGN=CENTER>

# Introduction to Text Data Format
## Professor Robert J. Brunner
  
</DIV>  
-----
-----

## Introduction

We have already touched upon data formats in the context of data
persistence. But one of the most important tasks when starting a data
analysis project is understanding the format of a data file and how to
best extract the necessary information from the data, whatever the
format. In this notebook, we explore different text data format, and present
how to read and write data in this format by using standard, built-in
Python tools.

-----



## Table of Contents

[XX](#XX-YY)

-----

Before proceeding with the rest of this notebook, we first have our standard notebook setup code.

-----

Before we begin, however, we need to read in test data to be able to
have data that we can write and read to different text based formats.
Since understanding the following code is actually one of the points of
this Notebook, first run the notebook, and return to this code cell
after completing the rest of the Notebook.

First, we make a temporary directory to store the data being processed before we 
grab the data of interest from the appropriate website. Next,
we read that data into a Python list for subsequent processing.

-----

In [1]:
# We delete the old directory first (assuming it exists)
# Before creaitng the new temp working directory
!rm -rf /home/data_scientist/temp
!mkdir /home/data_scientist/temp

In [2]:
# Grab the working data
!wget -O /home/data_scientist/temp/airports.csv http://stat-computing.org/dataexpo/2009/airports.csv

--2016-10-14 17:11:50--  http://stat-computing.org/dataexpo/2009/airports.csv
Resolving stat-computing.org (stat-computing.org)... 54.231.177.35
Connecting to stat-computing.org (stat-computing.org)|54.231.177.35|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244438 (239K) [text/csv]
Saving to: ‘/home/data_scientist/temp/airports.csv’

/home/data_scientis 100%[=====================>] 238.71K   357KB/s   in 0.7s   

2016-10-14 17:11:51 (357 KB/s) - ‘/home/data_scientist/temp/airports.csv’ saved [244438/244438]



In [3]:
import csv

airports = []

with open('/home/data_scientist/temp/airports.csv', 'r') as csvfile:
    
    for row in csv.reader(csvfile, delimiter=','):
        airports.append(row)

print(airports[0:3])

[['iata', 'airport', 'city', 'state', 'country', 'lat', 'long'], ['00M', 'Thigpen ', 'Bay Springs', 'MS', 'USA', '31.95376472', '-89.23450472'], ['00R', 'Livingston Municipal', 'Livingston', 'TX', 'USA', '30.68586111', '-95.01792778']]


-----
[[Back to TOC]](#Table-of-Contents)

## Formatted Text

The first data format we will discuss are formatted text files. These
types of files are seemingly ubiquitous, despite (or perhaps because of)
relying on a predominantly ASCII character encoding. This type of data
format actually comes in two varieties: fixed-width format, and
delimiter separated value format.

Fixed-width format files are preferred by some programming languages, for
example, Fortran, and also are easy to view when displayed. But they can
be non-trivial to decode in C-based programming languages, including
Python. To demonstrate a fixed width format, we can write and
subsequently read the airport data we extracted from the SQLite3
database. To write this data as a fixed width data format, we first need
to construct appropriate [string formatting codes][sfc]. 

In the following code cell, we construct two format code strings, the
first one is for the header row that contains the column labels, while
the second one is for the data rows. These format codes are fairly easy
to understand, we first enclose each string substitution in curly braces
`{ }`, and use numbers to indicate the order of substitution. Next, we
provide a colon `:` character to indicate the presence of a formatting
code, which consists of numbers and a letter code. The numbers following
the colon indicate the field width (in characters) and for
floating-point data, the numbers after the period specify the precision
(or numbers after the decimal point) of the value. The character code
indicates the type of data to encode: 's' for string, and 'f' for
floating-point. Thus, for example, `{1:29s}` means first format code
substitution, with a representation that is 29 characters wide that will
accept a string.

-----

[sfc]: https://docs.python.org/3.4/library/string.html#format-string-syntax

In [4]:
# First we define our format specification codes
hfmt = "{0:5s}{1:29s}{2:27s}{3:6s}{4:10s}{5:12s}{6:10s}\n"
fmt = "{0:5s}{1:29s}{2:30s}{3:3s}{4:4s}{5:14.8f}{6:14.8f}\n"

# We need to treat the first row special since it is the header row
flag = True

# Now open file and write out airports.
with open('/home/data_scientist/temp/fixed-width.txt', 'w') as fout:
    for row in airports:

        # We output first line special since it is a header row.
        if flag:
            fout.write(hfmt.format(row[0], row[1], row[2], row[3], row[4], \
                                   row[5], row[6]))
            flag = False
        
        else:
            fout.write(fmt.format(row[0], row[1], row[2], row[3], row[4], \
                                  float(row[5]), float(row[6])))

-----

After running the previous code cell, we have a new file called
`fixed-width.txt` that contains the list of airports in a fixed width
format file. We can first verify this by either using the Unix command
line tools to display the files contents, or simply use the `head`
command to see the first few lines.

-----

In [5]:
!head -5 /home/data_scientist/temp/fixed-width.txt

iata airport                      city                       state country   lat         long      
00M  Thigpen                      Bay Springs                   MS USA    31.95376472  -89.23450472
00R  Livingston Municipal         Livingston                    TX USA    30.68586111  -95.01792778
00V  Meadow Lake                  Colorado Springs              CO USA    38.94574889 -104.56989330
01G  Perry-Warsaw                 Perry                         NY USA    42.74134667  -78.05208056


-----

One of the hardest parts of dealing with fixed-width format data is
quantifying the actual widths of each column. As demonstrated by the
previous example, if you know the field widths ahead of time the
challenge is greatly simplified. However, for unknown data, simply
viewing the data is often not particularly helpful. In this case a good
approach is to print out the numerical column values above a few rows of
data to simplify determining the widths of each field. This is
demonstrated in the following code cell.

----

In [6]:
# Now lets quantify fixed width fields

# We print out the character markers

# First, we create a string of ten space characers, and join to that 
# strings of the numbers 1 - 9, each repeated ten times.

print ((' ' * 10) + ''.join([str(i)*10 for i in range(1, 10)]))

# The second line is simply a repeat of the digits 0-9 ten times, 
# followed by a line of dashes
print('0123456789'*10)
print('-'*100)

# We only want to output a few lines, so we count the rows
rCount = 0

# Now Read in file data.
with open('/home/data_scientist/temp/fixed-width.txt', 'r') as fin:
    
    for row in fin:
        
        # We skip last character, which is newline
        print(row[:-1])
        
        # We only want to print out first five rows.
        rCount += 1
        if rCount > 5: 
            break

          111111111122222222223333333333444444444455555555556666666666777777777788888888889999999999
0123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789
----------------------------------------------------------------------------------------------------
iata airport                      city                       state country   lat         long      
00M  Thigpen                      Bay Springs                   MS USA    31.95376472  -89.23450472
00R  Livingston Municipal         Livingston                    TX USA    30.68586111  -95.01792778
00V  Meadow Lake                  Colorado Springs              CO USA    38.94574889 -104.56989330
01G  Perry-Warsaw                 Perry                         NY USA    42.74134667  -78.05208056
01J  Hilliard Airpark             Hilliard                      FL USA    30.68801250  -81.90594389


-----

From this output, we can see the widths of each column. For example, the
first column extends from column 0 to column 4, while the second column
extends across columns 5-33, inclusive. With this information, we could
next read each row as a string, pull out the respective characters for
each field, strip any leading or trailing white space, and convert the
character data to the appropriate format. This code sample, however, is
left as an exercise for the breakout session. An alternative technique
would be to use Regular Expressions, discused in the [Introduction to
Data Parsing][idp] notebook to exactly match the requisite number of
characters and convert the results appropriately.

-----

[idp]: intro2dp.ipynb

-----
[[Back to TOC]](#Table-of-Contents)

### Delimiter Separated Values

A more popular formatted text data format is the delimiter separated
value format where a specific delimiter character is used to
distinguish between values in different columns. Traditionally, the
delimiter most frequently used is the comma, leading to the
comma-separated value (CSV) format. However, other delimiters are also
used, including whitespace characters like the space or tab characters,
or specific, infrequently used characters like the vertical bar `|`. 

We can easily read and write delimiter separated value formats by using
the read and rite methods in the [`csv` module][csv]. These methods
include an optional `delimiter` parameter that can be used to specify
the actual value to use to distinguish between consecutive values in
row. Other parameters can also be used to control how to escape the
delimiter character and how to indicate the end of a line.

-----
[csv]: https://docs.python.org/3/library/csv.html

In [7]:
# We will write a CSV file using the | character as a delimiter
import csv

with open('/home/data_scientist/temp/data.csv', 'w') as csvfile:
    
    # We need out csv writer stream
    fout = csv.writer(csvfile, delimiter='|')
    
    # Now write each airport out using the delimiter
    for airport in airports:
        fout.writerow(airport)

-----

This simple code block demonstrated how to write out a vertical-bar
separated value file. We can either view the file contents by using
Unix command line tools or by using the IPython Notebook, as
demonstrated in the next cell.

-----

In [8]:
!head -5 /home/data_scientist/temp/data.csv

----- 

Reading the data into a Python program is straightforward, simply use
the `csv.reader` method to iterate through the rows in the file. We
demonstrate this in the following code cell, where we convert the data
to fixed-width format to improve the readability of the resulting output.

-----

In [9]:
# We can read the data and display by usiung our previous string format codes.
hfmt = "{0:5s}{1:29s}{2:27s}{3:6s}{4:10s}{5:12s}{6:10s}"
fmt = "{0:5s}{1:29s}{2:30s}{3:3s}{4:4s}{5:14.8f}{6:14.8f}"

# First line is header row
rCount = 0

# Now Read in file data.
            
with open('/home/data_scientist/temp/data.csv', 'r') as csvfile:
    for row in csv.reader(csvfile, delimiter='|'):

        # We output first line special since it is a header row.
        if rCount == 0:
            print(hfmt.format(row[0], row[1], row[2], row[3], row[4], \
                              row[5], row[6]))
    
        # Else we simply print the row
        else:
            print(fmt.format(row[0], row[1], row[2], row[3], row[4], \
                             float(row[5]), float(row[6])))
        
        # We only want to print out first five rows.
        rCount += 1
        if rCount > 5: 
            break

iata airport                      city                       state country   lat         long      
00M  Thigpen                      Bay Springs                   MS USA    31.95376472  -89.23450472
00R  Livingston Municipal         Livingston                    TX USA    30.68586111  -95.01792778
00V  Meadow Lake                  Colorado Springs              CO USA    38.94574889 -104.56989330
01G  Perry-Warsaw                 Perry                         NY USA    42.74134667  -78.05208056
01J  Hilliard Airpark             Hilliard                      FL USA    30.68801250  -81.90594389


-----

<font color='red' size = '5'> Student Exercise </font>

Earlier in this notebook, we used XYZ. By using the preceding Code cells, try to make the following changes to see if your ability to identify outliers is improved.

3. Try 
4. Try adding 
2. Change the 

-----

## Ancillary Information

The following links are to additional documentation that you might find helpful in learning this material. Reading these web-accessible documents is completely optional.

4. A nice introduction on [][1]
1. An overview of [][2]
2. Wikipedia provides a [][3] 
43. Article discussing [][4] 

-----

[4]: http://

[1]: http://
[2]: https://
[3]: https://


**&copy; 2017: Robert J. Brunner at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode